In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [12]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [13]:
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic) #공백도 여기서는 하나의 원소

{"'": 0, 's': 1, 't': 2, 'b': 3, 'y': 4, 'o': 5, ',': 6, 'm': 7, 'u': 8, 'r': 9, 'l': 10, 'i': 11, 'e': 12, 'f': 13, 'k': 14, 'h': 15, 'g': 16, 'a': 17, 'd': 18, 'c': 19, ' ': 20, 'p': 21, 'w': 22, '.': 23, 'n': 24}


In [14]:
# 문자 집합 크기 확인
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [15]:
# 하이퍼 파라미터 설정
# 시퀸스 샘플 단위를 10개씩 끊음
hidden_size = dic_size
sequence_length = 10 # 임의 숫자 지정
learning_rate = 0.1

In [16]:
# 데이터 구성
# 시퀸스 랭스 값이 10인 데이터 단위로 샘플 잘라서 데이터 만듬
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)
                     
    x_data.append([char_dic[c] for c in x_str]) # x str to index
    y_data.append([char_dic[c] for c in y_str]) # y str to index
# 총 170개의 샘플 생성
# 각 샘플의 각 문자들은 고유한 정수로 인코딩됨

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [17]:
print(x_data[0])
print(y_data[0])

[11, 13, 20, 4, 5, 8, 20, 22, 17, 24]
[13, 20, 4, 5, 8, 20, 22, 17, 24, 2]


In [18]:
# 입력 시퀸스에 대해 원-핫 인코딩 수행
# 입력 데이터와 레이블 데이터를 텐서로 변환
x_one_hot = [np.eye(dic_size)[x] for x in x_data] # x데이터는 원-핫 인코딩
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터으 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터으 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


C:\Users\4lab\AppData\Local\Temp\ipykernel_100336\1747979889.py:4: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\Users\builder\tkoch\workspace\pytorch\pytorch_1647970138273\work\torch\csrc\utils\tensor_new.cpp:201.)
  X = torch.FloatTensor(x_one_hot)


In [19]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [20]:
print(Y[0])

tensor([13, 20,  4,  5,  8, 20, 22, 17, 24,  2])


In [21]:
# 모델 구현하기
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size 는 dic_size와 같음
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)
    
    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [22]:
net = Net(dic_size, hidden_size, 2)  # 층 두개

In [24]:
# nn.RNN()안에 num_layers 인자 사용
# 모델 선언 시 layers라는 인자에 2를 전달해 은닉층 2개 쌓음
# 비용함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [25]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환

torch.Size([1700, 25])


In [26]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [27]:
# 레이블 데이터는 (170, 10) 크기를 가짐
# 정확도 측정시 170*10으로 펼쳐서 1700의 크기를 가지고 측정하게 됨
# 옵티마이저와 손실함수 정의
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()
    
    # results 의 텐서 크기는 (170, 10)
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져옴
            predict_str += ''.join([char_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += char_set[result[-1]]
            
        print(predict_str)

m....sss.s
m....sss.sm
m....sss.sms
m....sss.smsm
m....sss.smsm.
m....sss.smsm.s
m....sss.smsm.sm
m....sss.smsm.sm.
m....sss.smsm.sm.s
m....sss.smsm.sm.s.
m....sss.smsm.sm.s..
m....sss.smsm.sm.s..s
m....sss.smsm.sm.s..s.
m....sss.smsm.sm.s..s.s
m....sss.smsm.sm.s..s.sm
m....sss.smsm.sm.s..s.sm.
m....sss.smsm.sm.s..s.sm..
m....sss.smsm.sm.s..s.sm...
m....sss.smsm.sm.s..s.sm....
m....sss.smsm.sm.s..s.sm.....
m....sss.smsm.sm.s..s.sm......
m....sss.smsm.sm.s..s.sm.......
m....sss.smsm.sm.s..s.sm.......s
m....sss.smsm.sm.s..s.sm.......sm
m....sss.smsm.sm.s..s.sm.......smm
m....sss.smsm.sm.s..s.sm.......smm.
m....sss.smsm.sm.s..s.sm.......smm..
m....sss.smsm.sm.s..s.sm.......smm...
m....sss.smsm.sm.s..s.sm.......smm....
m....sss.smsm.sm.s..s.sm.......smm.....
m....sss.smsm.sm.s..s.sm.......smm......
m....sss.smsm.sm.s..s.sm.......smm......m
m....sss.smsm.sm.s..s.sm.......smm......m.
m....sss.smsm.sm.s..s.sm.......smm......m.m
m....sss.smsm.sm.s..s.sm.......smm......m.m.
m....sss.smsm.sm.s..

e toe t nd
e toe t nd 
e toe t nd t
e toe t nd t 
e toe t nd t e
e toe t nd t e 
e toe t nd t e o
e toe t nd t e on
e toe t nd t e one
e toe t nd t e onet
e toe t nd t e onet 
e toe t nd t e onet t
e toe t nd t e onet t 
e toe t nd t e onet t e
e toe t nd t e onet t e 
e toe t nd t e onet t e e
e toe t nd t e onet t e e 
e toe t nd t e onet t e e d
e toe t nd t e onet t e e d 
e toe t nd t e onet t e e d t
e toe t nd t e onet t e e d t 
e toe t nd t e onet t e e d t t
e toe t nd t e onet t e e d t t 
e toe t nd t e onet t e e d t t t
e toe t nd t e onet t e e d t t t 
e toe t nd t e onet t e e d t t t e
e toe t nd t e onet t e e d t t t eh
e toe t nd t e onet t e e d t t t eh 
e toe t nd t e onet t e e d t t t eh  
e toe t nd t e onet t e e d t t t eh   
e toe t nd t e onet t e e d t t t eh   t
e toe t nd t e onet t e e d t t t eh   t 
e toe t nd t e onet t e e d t t t eh   t e
e toe t nd t e onet t e e d t t t eh   t e 
e toe t nd t e onet t e e d t t t eh   t e  
e toe t nd t e onet 

p t ertand
p t ertand 
p t ertand t
p t ertand to
p t ertand to 
p t ertand to c
p t ertand to cu
p t ertand to cut
p t ertand to cutp
p t ertand to cutpt
p t ertand to cutpt 
p t ertand to cutpt t
p t ertand to cutpt to
p t ertand to cutpt tot
p t ertand to cutpt toto
p t ertand to cutpt totoe
p t ertand to cutpt totoep
p t ertand to cutpt totoepl
p t ertand to cutpt totoepl 
p t ertand to cutpt totoepl t
p t ertand to cutpt totoepl to
p t ertand to cutpt totoepl toe
p t ertand to cutpt totoepl toem
p t ertand to cutpt totoepl toemt
p t ertand to cutpt totoepl toemt 
p t ertand to cutpt totoepl toemt t
p t ertand to cutpt totoepl toemt to
p t ertand to cutpt totoepl toemt to 
p t ertand to cutpt totoepl toemt to m
p t ertand to cutpt totoepl toemt to m 
p t ertand to cutpt totoepl toemt to m t
p t ertand to cutpt totoepl toemt to m te
p t ertand to cutpt totoepl toemt to m tee
p t ertand to cutpt totoepl toemt to m teet
p t ertand to cutpt totoepl toemt to m teeto
p t ertand to cutpt 

p tosdland
p tosdland 
p tosdland t
p tosdland to
p tosdland to 
p tosdland to b
p tosdland to bu
p tosdland to bui
p tosdland to buil
p tosdland to buile
p tosdland to buile 
p tosdland to buile a
p tosdland to buile an
p tosdland to buile ans
p tosdland to buile ansh
p tosdland to buile anshi
p tosdland to buile anship
p tosdland to buile anship,
p tosdland to buile anship, 
p tosdland to buile anship, d
p tosdland to buile anship, do
p tosdland to buile anship, dos
p tosdland to buile anship, dos'
p tosdland to buile anship, dos't
p tosdland to buile anship, dos't 
p tosdland to buile anship, dos't d
p tosdland to buile anship, dos't dr
p tosdland to buile anship, dos't dru
p tosdland to buile anship, dos't drum
p tosdland to buile anship, dos't drum 
p tosdland to buile anship, dos't drum t
p tosdland to buile anship, dos't drum tp
p tosdland to buile anship, dos't drum tp 
p tosdland to buile anship, dos't drum tp p
p tosdland to buile anship, dos't drum tp po
p tosdland to buile 

d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to lo
d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to lon
d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to long
d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to long 
d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to long f
d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to long fo
d toodwant to build a ship, don't drum tp people together to bollect wood and don't dssign them tosks and work, but rather te ch them to long for

  too want
  too want 
  too want t
  too want to
  too want to 
  too want to b
  too want to bu
  too want to bui
  too want to buil
  too want to build
  too want to build 
  too want to build a
  too want to build a 
  too want to build a s
  too want to build a sh
  too want to build a shi
  too want to build a ship
  too want to build a ship,
  too want to build a ship, 
  too want to build a ship, d
  too want to build a ship, do
  too want to build a ship, don
  too want to build a ship, don'
  too want to build a ship, don't
  too want to build a ship, don't 
  too want to build a ship, don't d
  too want to build a ship, don't dr
  too want to build a ship, don't dru
  too want to build a ship, don't drum
  too want to build a ship, don't drum 
  too want to build a ship, don't drum u
  too want to build a ship, don't drum up
  too want to build a ship, don't drum up 
  too want to build a ship, don't drum up p
  too want to build a ship, don't drum up pe
  too want to build 

d you want
d you want 
d you want t
d you want to
d you want to 
d you want to b
d you want to bu
d you want to bui
d you want to buil
d you want to build
d you want to build 
d you want to build a
d you want to build a 
d you want to build a s
d you want to build a sh
d you want to build a shi
d you want to build a ship
d you want to build a ship,
d you want to build a ship, 
d you want to build a ship, d
d you want to build a ship, do
d you want to build a ship, don
d you want to build a ship, don'
d you want to build a ship, don't
d you want to build a ship, don't 
d you want to build a ship, don't a
d you want to build a ship, don't ar
d you want to build a ship, don't aru
d you want to build a ship, don't arum
d you want to build a ship, don't arum 
d you want to build a ship, don't arum u
d you want to build a ship, don't arum up
d you want to build a ship, don't arum up 
d you want to build a ship, don't arum up p
d you want to build a ship, don't arum up pe
d you want to build 

p you want
p you want 
p you want t
p you want to
p you want to 
p you want to b
p you want to bu
p you want to bui
p you want to buil
p you want to build
p you want to build 
p you want to build a
p you want to build a 
p you want to build a s
p you want to build a sh
p you want to build a shi
p you want to build a ship
p you want to build a ship,
p you want to build a ship, 
p you want to build a ship, d
p you want to build a ship, do
p you want to build a ship, don
p you want to build a ship, don'
p you want to build a ship, don't
p you want to build a ship, don't 
p you want to build a ship, don't d
p you want to build a ship, don't dr
p you want to build a ship, don't dru
p you want to build a ship, don't drum
p you want to build a ship, don't drum 
p you want to build a ship, don't drum u
p you want to build a ship, don't drum up
p you want to build a ship, don't drum up 
p you want to build a ship, don't drum up p
p you want to build a ship, don't drum up pe
p you want to build 

f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the e
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the en
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the end
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endl
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endle
f you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endles
f you want to build a ship, don't drum up people together to collect wood

p you want
p you want 
p you want t
p you want to
p you want to 
p you want to b
p you want to bu
p you want to bui
p you want to buil
p you want to build
p you want to build 
p you want to build a
p you want to build a 
p you want to build a s
p you want to build a sh
p you want to build a shi
p you want to build a ship
p you want to build a ship,
p you want to build a ship, 
p you want to build a ship, d
p you want to build a ship, do
p you want to build a ship, don
p you want to build a ship, don'
p you want to build a ship, don't
p you want to build a ship, don't 
p you want to build a ship, don't d
p you want to build a ship, don't dr
p you want to build a ship, don't dru
p you want to build a ship, don't drum
p you want to build a ship, don't drum 
p you want to build a ship, don't drum u
p you want to build a ship, don't drum up
p you want to build a ship, don't drum up 
p you want to build a ship, don't drum up p
p you want to build a ship, don't drum up pe
p you want to build 